In [1]:
# importing libraries
import pandas as pd
import csv
from datetime import datetime
import matplotlib.pyplot as plt
%run data_entry.ipynb

In [13]:
# implementing class for more interactivity with the tracker
class CSV:
    CSV_FILE= "finance_data.csv"
    COLUMNS=["date","amount","category","description"]
    FORMAT= "%d-%m-%Y"
    @classmethod
    def initialize_csv(cls):
        try:
            pd.read_csv(cls.CSV_FILE)
        except:
            df=pd.DataFrame(columns=cls.COLUMNS)
            df.to_csv(cls.CSV_FILE,index=False)

# instance for data entry    
    @classmethod
    def add_entry(cls,date,amount,category,description):
        new_entry={
            "date":date,
            "amount":amount,
            "category":category,
            "description":description
        }
        with open(cls.CSV_FILE,"a",newline="") as csvfile:
            writer=csv.DictWriter(csvfile,fieldnames=cls.COLUMNS)
            writer.writerow(new_entry)
        print("Entry added sucessfully")

# instance to get transaction details
    @classmethod
    def get_transactions(cls,start_date,end_date):
        df=pd.read_csv(cls.CSV_FILE)
        df["date"] = pd.to_datetime(df["date"],format=CSV.FORMAT)
        start_date= datetime.strptime(start_date,CSV.FORMAT)
        end_date= datetime.strptime(end_date,CSV.FORMAT)
        
        mask=(df["date"]>=start_date) & (df["date"]<= end_date)
        filtered_df=df.loc[mask]

        if filtered_df.empty:
            print("Not transaction is found in given date range")
        else:
            print(f"Transactions from {start_date.strftime(CSV.FORMAT)} to {end_date.strftime(CSV.FORMAT)}")
            print(filtered_df.to_string(index=False, formatters={"date":lambda x:x.strftime(CSV.FORMAT)}))
            total_income= filtered_df[filtered_df["category"]=="Income"]["amount"].sum()
            total_expense=filtered_df[filtered_df["category"]=="Expense"]["amount"].sum()
            print("\nSummary:")
            print(f"Total Income: Rs.{total_income:.2f}")
            print(f"Total Expense: Rs.{total_expense:.2f}")
            print(F"Net Savings: Rs.{(total_income-total_expense):.2f}")

        return filtered_df

#  adding new data
def add():
    CSV.initialize_csv()
    date=get_date("Enter the date of the transaction (dd-mm-yyyy) or enter for today's date: ",allow_default=True)
    amount=get_amount()
    category=get_category()
    description=get_desription()
    CSV.add_entry(date, amount, category, description)

CSV.get_transactions("01-01-2023","30-07-2024")

# ploting the graph
def plot_transactions(df):
    df.set_index('date',inplace=True)
    income_df=df[df["category"]=="Income"].resample("D").sum().reindex(df.index, fill_value=0)
    expense_df=df[df["category"]=="Expense"].resample("D").sum().reindex(df.index, fill_value=0)
    plt.figure(figsize=(7,4))
    plt.plot(income_df.index,income_df["amount"],label="Income",color="g")
    plt.plot(expense_df.index,expense_df["amount"],label="Expense",color="r")
    plt.xlabel("Date",fontsize=10)
    plt.ylabel("Amount",fontsize=10)
    plt.title("Income and Expense over Time",fontsize=12)
    plt.legend()
    plt.grid(True)
    plt.show()

# calling all the functions
def main():
    while True:
        print("\n1. Add a new transaction")
        print("2. View transactions and summary within a date range")
        print("3. Exit\n")
        choice=input("Enter your choice (1-3): ")

        if choice == "1":
            add()
        elif choice == "2":
            start_date = get_date("Enter the start date (dd-mm-yyyy): ")
            end_date = get_date("Enter the end date (dd-mm-yyyy): ")
            df=CSV.get_transactions(start_date,end_date)
            if input("Do you want to see a plot? (y/n) ").lower()=="y":
                plot_transactions(df)
        elif choice=="3":
            print("Exiting...")
            break
        else:
            print("Invalid choice. Enter 1, 2 or 3.")

# doesn't run automatically unless we call it
if __name__=="__main__":
    main()

Not transaction is found in given date range

1. Add a new transaction
2. View transactions and summary within a date range
3. Exit



Enter your choice (1-3):  3


Exiting...
